<!--NOTEBOOK_HEADER-->
*This notebook contains course material from [CBE40455](https://jckantor.github.io/CBE40455) by
Jeffrey Kantor (jeff at nd.edu); the content is available [on Github](https://github.com/jckantor/CBE40455.git).
The text is released under the [CC-BY-NC-ND-4.0 license](https://creativecommons.org/licenses/by-nc-nd/4.0/legalcode),
and code is released under the [MIT license](https://opensource.org/licenses/MIT).*

<!--NAVIGATION-->
< [Newsvendor Problem](http://nbviewer.jupyter.org/github/jckantor/CBE40455/blob/master/notebooks/06.01-Newsvendor-Problem.ipynb) | [Contents](toc.ipynb) | [Risk Neutral Gambler](http://nbviewer.jupyter.org/github/jckantor/CBE40455/blob/master/notebooks/06.03-Risk-Neutral-Gambler.ipynb) ><p><a href="https://colab.research.google.com/github/jckantor/CBE40455/blob/master/notebooks/06.02-Scenario-Analysis-for-a-Plant-Expansion.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>

# Scenario Analysis for a Plant Expansion

In [54]:
%%writefile tmp/PlantExpansion.mod

set PLANTS;                            # Set of plant types
set DEMAND;                            # Demand Segments
set SCENARIOS;                         # Planning Scenarios

param e{PLANTS};                       # Current Plant Capacity
param C{PLANTS};                       # Capital Cost per unit Expansion
param O{PLANTS};                       # Operating Cost [k$/GWh]

param T{DEMAND};                       # Time Periods for Demand Segments
param D{DEMAND,SCENARIOS};             # Demand Scenarios

var x {PLANTS} >= 0;                   # Plant Expansion
var y {PLANTS,DEMAND,SCENARIOS} >= 0;  # Operating Schedule
var v {SCENARIOS};                     # Variable Cost
var capcost;                           # Capital Cost

minimize COST: capcost + sum {s in SCENARIOS} 0.25*v[s];

s.t. CAPCOST: capcost = sum{p in PLANTS} C[p]*(e[p]+x[p]);
s.t. VARCOST {s in SCENARIOS}:
   v[s] = sum {p in PLANTS, d in DEMAND} T[d]*O[p]*y[p,d,s];
s.t. DEMANDS {p in PLANTS, s in SCENARIOS}: 
   e[p] + x[p] >= sum {d in DEMAND} y[p,d,s];
s.t. C4 {d in DEMAND, s in SCENARIOS} :
   D[d,s] = sum {p in PLANTS} y[p,d,s];
   
solve;

table results {p in PLANTS} OUT "CSV" "tmp/PlantExpansion.csv" "Table" :
    p~Plant,
    O[p]~Unit_Cost,
    e[p]~Current_Cap,
    x[p]~Exp_Cap,
    x[p]+e[p]~Total_Cap;

end;


Overwriting tmp/PlantExpansion.mod


In [55]:
%%script glpsol -m tmp/PlantExpansion.mod -d /dev/stdin -y tmp/PlantExpansion.txt --out output

set SCENARIOS := S1 S2 S3 S4;

param: DEMAND: T :=
    Base      24
    Peak       6 ;

param: PLANTS:     e     C     O:=
    Coal        1.75   200    30
    Hydro       2.00   500    10
    Nuclear     0.00   300    20
    Grid        0.00     0   200 ;

param D :   S1     S2    S3    S4 :=
    Base   8.25   10.0  7.50  9.00
    Peak   2.50   2.00  2.50  1.50 ;

end;

In [56]:
print output

GLPSOL: GLPK LP/MIP Solver, v4.52
Parameter(s) specified in the command line:
 -m tmp/PlantExpansion.mod -d /dev/stdin -y tmp/PlantExpansion.txt
Reading model section from tmp/PlantExpansion.mod...
tmp/PlantExpansion.mod:37: warning: final NL missing before end of file
37 lines were read
Reading data section from /dev/stdin...
18 lines were read
Generating COST...
Generating CAPCOST...
Generating VARCOST...
Generating DEMANDS...
Generating C4...
Model has been successfully generated
GLPK Simplex Optimizer, v4.52
30 rows, 41 columns, 125 non-zeros
Preprocessing...
24 rows, 36 columns, 80 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 24
      0: obj =   4.560000000e+04  infeas =  4.325e+01 (0)
*     4: obj =   4.560000000e+04  infeas =  0.000e+00 (0)
*    28: obj =   7.573750000e+03  infeas =  0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 sec

In [57]:
import pandas

results = pandas.read_csv("tmp/PlantExpansion.csv")
results

,Plant,Unit_Cost,Current_Cap,Exp_Cap,Total_Cap
0,Coal,30,1.75,0.25,2.0
1,Hydro,10,2.00,5.50,7.5
2,Nuclear,20,0.00,2.50,2.5
3,Grid,200,0.00,0.00,0.0


<!--NAVIGATION-->
< [Newsvendor Problem](http://nbviewer.jupyter.org/github/jckantor/CBE40455/blob/master/notebooks/06.01-Newsvendor-Problem.ipynb) | [Contents](toc.ipynb) | [Risk Neutral Gambler](http://nbviewer.jupyter.org/github/jckantor/CBE40455/blob/master/notebooks/06.03-Risk-Neutral-Gambler.ipynb) ><p><a href="https://colab.research.google.com/github/jckantor/CBE40455/blob/master/notebooks/06.02-Scenario-Analysis-for-a-Plant-Expansion.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>